In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, f1_score, precision_score
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix as cm


filterwarnings('ignore')

In [ ]:
df = pd.read_csv("../input/tabletcsv/Tablet.csv")

Veri setinin hedef değişkeninin dengeli dağılıp dağılmadığı incelenmeli ve yorumlanmalıdır

In [ ]:
df["FiyatAraligi"].value_counts()

**Korelasyon matrisleri çizdirilmeli ve anlamlı görünen değişkenler arasındaki korelasyon katsayıları yorumlanmalıdır.**

In [ ]:
df.corr()

**Veri setindeki sayısal değişkenlerin ortalama, medyan, standart sapma gibi değerleri görüntülenmeli ve yorumlanmalıdır.**

In [ ]:
df.mean()

In [ ]:
df.std() 

In [ ]:
df.median()

Sayısal ve kategorik değişkenler ideal tekniklerle ve araçlarla görselleştirilmeli ve önemli olduğu düşünülen kısımlar yorumlanmalıdır.

In [ ]:
sns.scatterplot(x = "BataryaGucu", y = "DahiliBellek", hue = "FiyatAraligi", data = df);

In [ ]:
sns.countplot("Renk", data = df);

In [ ]:
sns.countplot("FiyatAraligi", data = df);

In [ ]:
sns.factorplot("CiftHat", "DahiliBellek", "4G", data = df, kind = "bar");

**Ön işleme kapsamında eğer veri setinde eksik veriler varsa bu eksik verilerin mantıklı gerekçelerle doldurulması beklenmektedir. Bu işlem esnasında hedef değişkenin anlamlılığını kaybetmemesi için eksik verilerin hedef değişken özelinde incelenip doldurulması beklenmektedir. Ayrıca bu safhada kategorik değişkenlerin sıralanabilir(ordered) veya sıralanamaz(nominal) olma durumlarına göre uygun sayısallaştırma işlemleri yapılmalıdır.**

In [ ]:
df.isna().sum()

In [ ]:
df["OnKameraMP"].mean()

In [ ]:
df["RAM"].mean()

In [ ]:
df["OnKameraMP"].fillna(4.312280701754386,inplace=True)
df["OnKameraMP"]

In [ ]:
df["RAM"].fillna(2116.243963782696,inplace=True)
df["RAM"]

In [ ]:
df.isna().sum()

**Hedef değişkenimiz olan “FiyatAraligi” değişkeni ve bu değişkeni tahmin etmemize yardımcı olacak diğer değişkenler bağımlı ve bağımsız değişkenler olmak üzere ayrılmalıdır.** 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
df["Bluetooth"] = le.fit_transform(df["Bluetooth"]) #1 ise yes, 0 ise No

In [ ]:
df["CiftHat"] = le.fit_transform(df["CiftHat"])

In [ ]:
df["Dokunmatik"] = le.fit_transform(df["Dokunmatik"])

In [ ]:
df["WiFi"] = le.fit_transform(df["WiFi"])

In [ ]:
df["3G"] = le.fit_transform(df["3G"])

In [ ]:
df["4G"] = le.fit_transform(df["4G"])

In [ ]:
df_region=pd.get_dummies(df.Renk, prefix='Renk')

In [ ]:
df_region

In [ ]:
df["Renk"]=df_region

In [ ]:
df = pd.concat([df, df_region], axis = 1)
df

In [ ]:
X = df.drop("FiyatAraligi", axis = 1)
y = df["FiyatAraligi"]

In [ ]:
X

In [ ]:
y

**Sonrasında bağımlı ve bağımsız değişkenler %85 oranında eğitim ve %15 oranında test olmak üzere “train_test_split” metodu kullanılarak ayrılmalıdır.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

**Eğitim kümesi ile GaussianNB, DecisionTree ve KNN modelleri eğitilmelidir**

1) DecisionTree modeli

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='gini')
cart_model=cart.fit(X_train,y_train)

In [ ]:
cart_model

2) KNN Modeli

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

3) GaussianNB Modeli

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

**Doğruluk skoru elde edilerek modeller arasındaki başarı durumu karşılaştırma yapılmalıdır**. 

1) DecisionTree modeli doğruluk skoru hesaplama

In [ ]:
y_pred = cart_model.predict(X_test)
accuracy_score(y_test, y_pred)

2) KNN modeli doğruluk skoru hesaplama

In [ ]:
y_pred2 = knn_model.predict(X)
accuracy_score(y, y_pred2)

3) GaussianNB doğruluk skoru hesaplama

In [ ]:
y_pred3 = model.predict(X_test)
accuracy_score(y_test,y_pred3)

**Her bir model için karmaşıklık matrisi çizdirilmelidir.**

1) DecisionTree modeli karmaşık matrisi

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

2) KNN modeli karmaşık matrisi

In [ ]:
karmasiklik_matrisi2 = confusion_matrix(y, y_pred2)
print(karmasiklik_matrisi2)

3) GaussianNB modeli karmaşık matrisi

In [ ]:
karmasiklik_matrisi3 = confusion_matrix(y_test, y_pred3)
print(karmasiklik_matrisi3)

  **Modellerin başarısını değerlendirme konusunda F1_Score, Precision, Recall değerleri “classification_report” aracılığıyla yazdırılmalıdır.**

1) DecisionTree modeli

In [ ]:
print(classification_report(y_test, y_pred))

2) KNN modeli 

In [ ]:
print(classification_report(y, y_pred2))

3) GaussianNB modeli 

In [ ]:
print(classification_report(y_test, y_pred3))

**Ekip lideri DecisionTree algoritmasının varsayılan olarak (criterion = “gini”) parametre değerini kullandığını ancak bu problem için criterion parametresini (criterion = “entropy”) olacak şekilde değiştirdiklerinde başarının artacağı konusunda ekibi temin etmiştir**

In [ ]:
cart= DecisionTreeClassifier(random_state = 42, criterion='gini')
cart_model = cart.fit(X_train, y_train)

In [ ]:
y_pred5 = cart_model.predict(X_test)
accuracy_score(y_test, y_pred5)

In [ ]:
cart2 = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model2=cart2.fit(X_train,y_train)

In [ ]:
y_pred6 = cart_model2.predict(X_test)
accuracy_score(y_test, y_pred6)

**KNN algoritması konusunda komşu sayısının çok kritik olduğunu ve 2’den 50’ye kadar olan her sayıyı komşu sayısı olarak deneyerek her denemede modelin skorunu kaydetmeyi önermektedir.**

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
knn_model

In [ ]:
knn = range(2,50)
for k_degeri in knn:
    knn = KNeighborsClassifier(n_neighbors = k_degeri)
    knn_model = knn.fit(X_train, y_train)
    skor = knn_model.score(X_test, y_test)
    print("k=", k_degeri ,"degeri için =>", skor)

**Komşu sayısının değişimi ile modelin skorunun da değişeceğini ve her komşu sayısına tekabül eden model skorunu bir plot çizdirerek görselleştirme yapılması gerektiğini rica etmiştir.**

In [ ]:
skor_liste = []

for each in range(2,50,1):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    skor_liste.append(knn2.score(X_test, y_test))

plt.plot(range(2,50,1),skor_liste)
plt.xlabel("k en yakın komşu sayıları")
plt.ylabel("Skor")
plt.show()